<a href="https://colab.research.google.com/github/yichen917/Kaggle_ML_Projects_M5_Forecasting_Accuracy/blob/main/DL_ALL_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### LSTM for all data
This notebook is the code for our training and prediction. Since we divided the dataset into three files based on product categories (HOBBIES, FOODS, HOUSEHOLD), we will execute this notebook three times and then merge the output prediction results to obtain the final prediction file.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras.callbacks import Callback, ModelCheckpoint

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [ ]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
HH_data = pd.read_csv("/content/drive/My Drive/統計學習初論/Editted data/HOUSEHOLD_df.csv",\
                      dtype = {'item_id':'str', 'dept_id':'str', 'store_id':'str', 'month':'str', 'year':'str', 'snap':'str'})

In [ ]:
test_1_data = pd.read_csv("/content/drive/My Drive/統計學習初論/Editted data/test_1_HOUSEHOLD_df.csv",\
                      dtype = {'item_id':'str', 'dept_id':'str', 'store_id':'str', 'month':'str', 'year':'str', 'snap':'str'})

In [ ]:
test_2_data = pd.read_csv("/content/drive/My Drive/統計學習初論/Editted data/test_2_HOUSEHOLD_df.csv",\
                      dtype = {'item_id':'str', 'dept_id':'str', 'store_id':'str', 'month':'str', 'year':'str', 'snap':'str'})

In [ ]:
HH_data.head(2)

,item_id,dept_id,cat_id,store_id,state_id,num_sold,weekday,month,year,snap,sell_price,d_num,event_type_1_National,event_type_1_Religious,event_type_1_Sporting,dept_item
0,001,1,HOUSEHOLD,1,CA,3,Saturday,1,2011,0,6.32,1,0,0,0,1_001
1,001,1,HOUSEHOLD,1,CA,0,Sunday,1,2011,0,6.32,2,0,0,0,1_001


In [ ]:
test_1_data.head(2)

,item_id,dept_id,cat_id,store_id,state_id,num_sold,weekday,month,year,snap,sell_price,d_num,event_type_1_Religious,event_type_1_National,event_type_1_Sporting,dept_item
0,001,1,HOUSEHOLD,1,CA,1,Monday,4,2016,0,5.97,1914,0,0,0,1_001
1,001,1,HOUSEHOLD,1,CA,0,Tuesday,4,2016,0,5.97,1915,0,0,0,1_001


In [ ]:
test_2_data.head(2)

,item_id,dept_id,cat_id,store_id,state_id,weekday,month,year,snap,sell_price,d_num,event_type_1_Religious,event_type_1_Sporting,event_type_1_National,dept_item
0,001,1,HOUSEHOLD,1,CA,Monday,4,2016,0,5.97,1942,0,0,0,1_001
1,001,1,HOUSEHOLD,1,CA,Tuesday,4,2016,0,5.97,1943,0,0,0,1_001


In [ ]:
data = HH_data

In [ ]:
def create_id_col(data):
  id_col = data['cat_id'] + '_' + data['dept_id'] + '_' + data['item_id'] + '_' + data['state_id'] + '_' + data['store_id']
  return id_col

In [ ]:
# Create id
data['id'] = create_id_col(data)
test_1_data['id'] = create_id_col(test_1_data)
test_2_data['id'] = create_id_col(test_2_data)

In [ ]:
data.head()

,item_id,dept_id,cat_id,store_id,state_id,num_sold,weekday,month,year,snap,sell_price,d_num,event_type_1_National,event_type_1_Religious,event_type_1_Sporting,dept_item,id
0,001,1,HOUSEHOLD,1,CA,3,Saturday,1,2011,0,6.32,1,0,0,0,1_001,HOUSEHOLD_1_001_CA_1
1,001,1,HOUSEHOLD,1,CA,0,Sunday,1,2011,0,6.32,2,0,0,0,1_001,HOUSEHOLD_1_001_CA_1
2,001,1,HOUSEHOLD,1,CA,0,Monday,1,2011,0,6.32,3,0,0,0,1_001,HOUSEHOLD_1_001_CA_1
3,001,1,HOUSEHOLD,1,CA,0,Tuesday,2,2011,1,6.32,4,0,0,0,1_001,HOUSEHOLD_1_001_CA_1
4,001,1,HOUSEHOLD,1,CA,0,Wednesday,2,2011,1,6.32,5,0,0,0,1_001,HOUSEHOLD_1_001_CA_1


In [ ]:
print(data.shape)
print(test_1_data.shape)
print(test_2_data.shape)

(15989319, 17)
(293160, 17)
(293160, 16)


In [ ]:
def create_xy_data(df, pre_type = ""): # for one item at once
  x_train_id = (df['id'] + "_" + pre_type).values
  idx = np.unique(x_train_id, return_index=True)[1]
  idx.sort()
  x_train_id = x_train_id[idx]
  y_train = df['num_sold'].values
  df = df.drop(['id','d_num','num_sold','item_id', 'dept_id', 'cat_id', 'dept_item','year'],axis=1)
  return df, y_train, x_train_id

In [ ]:
def create_x(df, pre_type=""): # for one item at once
  x_train_id = (df['id'] + "_" + pre_type).values
  idx = np.unique(x_train_id, return_index=True)[1]
  idx.sort()
  x_train_id = x_train_id[idx]
  df = df.drop(['id','d_num','item_id', 'dept_id', 'cat_id', 'dept_item','year'],axis=1)
  return df, x_train_id

In [ ]:
import keras
from keras import Sequential
from keras import layers

In [ ]:
def build_model(shape):
  model = Sequential()
  model.add(layers.LSTM(50, input_shape=(shape[1], shape[2]), return_sequences=True))
  model.add(layers.LSTM(40))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(16, activation='relu'))
  model.add(layers.Dense(8, activation='relu'))
  model.add(layers.Dense(1, activation = 'linear'))

  model.compile(
      loss= keras.losses.MeanSquaredError(),
      optimizer='adam',
  )

  return model

In [ ]:
# According to the autoregression results, the sales of the current day are highly correlated with the sales of the previous 1, 7, and 14 days.
# Therefore, we choose these days as the number of days to look back in our data.
LOOKBACK_MAX = 14
LOOKBACK_ARR = np.array([0,1,7,14])

y_val_pre_all = []
y_eval_pre_all = []
val_id_all = []
eval_id_all = []
SE_list = []      # list for mean squared error of each item

# We create a model for each product (unique dept_item_id), and move on to the next one after predicting one product.
for dept_item_id in dept_item_ids: # one item at once
  print("-----------------------------------")
  print("Current item is ", dept_item_id)
  train_data = data[data['dept_item'] == dept_item_id].reset_index(drop=True)
  val_data = test_1_data[test_1_data['dept_item'] == dept_item_id].reset_index(drop=True)
  eval_data = test_2_data[test_2_data['dept_item'] == dept_item_id].reset_index(drop=True)

  # Since it is not reasonable to use data from different stores when extracting the previous 1, 7, 14 days, we save the store names for later use.
  state_store_list = (train_data['state_id'] + '_' + train_data['store_id']).to_numpy()
  val_state_store_list = (val_data['state_id'] + '_' + val_data['store_id']).to_numpy()
  eval_state_store_list = (eval_data['state_id'] + '_' + eval_data['store_id']).to_numpy()

  # Transform categorical data using OHE
  train_dummy = pd.get_dummies(train_data,columns = ['store_id','state_id', 'weekday', 'snap'], drop_first=True)
  train_columns = train_dummy.columns
  val_dummy = pd.get_dummies(val_data,columns = ['store_id','state_id', 'weekday', 'snap'], drop_first=True)
  eval_dummy = pd.get_dummies(eval_data,columns = ['store_id','state_id', 'weekday', 'snap'], drop_first=True)

  # Create dataframe
  x_data, y_data, data_id = create_xy_data(train_dummy)
  x_val, y_val, val_id = create_xy_data(val_dummy, pre_type="validation")
  x_eval, eval_id = create_x(eval_dummy, pre_type="evaluation")

  val_id_all.append(val_id)
  eval_id_all.append(eval_id)

  # Standardize price
  scaler_x = StandardScaler()
  x_data['sell_price'] = scaler_x.fit_transform(np.array(x_data['sell_price']).reshape(-1,1))
  x_val['sell_price'] = scaler_x.transform(np.array(x_val['sell_price']).reshape(-1,1))
  x_eval['sell_price'] = scaler_x.transform(np.array(x_eval['sell_price']).reshape(-1,1))
  x_columns = x_data.columns


  # Create the three-dimensional dataframe including lookback days
  x_time = []
  y_time = []
  x_val_time = []
  y_val_time = []
  x_eval_time = []
  i = LOOKBACK_MAX
  current_store = state_store_list[i]

  while i < len(x_data):
    i_store = state_store_list[i]
    if  i_store != current_store or i == len(x_data)-1: # encounter a new store or is the last store
      train_last = x_data.iloc[i-LOOKBACK_MAX:i]
      val_index = np.where(val_state_store_list == current_store)[0]
      val_store = x_val.iloc[val_index]
      y_val_time.append(y_val[val_index])

      eval_index = np.where(eval_state_store_list == current_store)[0]
      eval_store = x_eval.iloc[eval_index]
      combine_df = pd.concat([train_last, val_store, eval_store])

      j = LOOKBACK_MAX
      while j < 28+LOOKBACK_MAX:
        x_val_time.append(combine_df.iloc[j-LOOKBACK_ARR].values.tolist())
        j += 1

      while j < len(combine_df):
        x_eval_time.append(combine_df.iloc[j-LOOKBACK_ARR].values.tolist())
        j += 1

      if i != len(x_data)-1:
        i += LOOKBACK_MAX
      current_store = i_store

    x_time.append(x_data.iloc[i-LOOKBACK_ARR].values.tolist())
    y_time.append(y_data[i])
    i += 1
  x_time = np.array(x_time)
  y_time = np.array(y_time)
  x_val_time = np.array(x_val_time)
  y_val_time = np.array(y_val_time)
  y_val_time = y_val_time.reshape(y_val_time.shape[0] * y_val_time.shape[1])
  x_eval_time = np.array(x_eval_time)

  # Fit the model. Since the loss did not decrease significantly after 5-10 epochs, we set the epochs at 5 to accelerate model training.
  model = build_model(x_time.shape)
  history = model.fit(x = x_time,
                      y = y_time,
                      epochs=5,
                      shuffle=True,
                      batch_size=128,
                      validation_split = 0.1,
                      verbose=1)
  # Predict
  y_val_pre = model.predict(x_val_time).reshape(-1,) # (280,)
  y_val_pre_all.append(y_val_pre)
  MSE = mean_squared_error(y_val_pre, y_val_time)
  SE_list.append(MSE*len(y_val_pre))
  print("RMSE = ", np.sqrt(MSE))

  y_eval_pre = model.predict(x_eval_time).reshape(-1,) #(280,)
  y_eval_pre_all.append(y_eval_pre)

Streaming output truncated to the last 5000 lines.
RMSE =  4.691327512158347
-----------------------------------
Current item is  1_180
Train on 12542 samples, validate on 1394 samples
Epoch 1/5
12542/12542 [==============================] - 2s 167us/step - loss: 3.3370 - val_loss: 1.8167
Epoch 2/5
12542/12542 [==============================] - 1s 113us/step - loss: 2.7630 - val_loss: 2.3189
Epoch 3/5
12542/12542 [==============================] - 1s 113us/step - loss: 2.6760 - val_loss: 2.8027
Epoch 4/5
12542/12542 [==============================] - 1s 113us/step - loss: 2.6446 - val_loss: 2.7833
Epoch 5/5
12542/12542 [==============================] - 1s 112us/step - loss: 2.5954 - val_loss: 2.7756
RMSE =  1.732768774949897
-----------------------------------
Current item is  1_181
Train on 17091 samples, validate on 1899 samples
Epoch 1/5
17091/17091 [==============================] - 3s 161us/step - loss: 0.7880 - val_loss: 0.4027
Epoch 2/5
17091/17091 [============================

#### Transform predicted results into the requested format

In [ ]:
y_val_pre_all = np.array(y_val_pre_all)
y_eval_pre_all = np.array(y_eval_pre_all)

y_val_output = y_val_pre_all.reshape(-1, 28)
y_eval_output = y_eval_pre_all.reshape(-1, 28)
print(y_val_output.shape,y_eval_output.shape)

(5320, 28) (5320, 28)


In [ ]:
val_id_all = np.array(val_id_all)
eval_id_all = np.array(eval_id_all)

val_id_output = val_id_all.reshape(val_id_all.shape[0]*val_id_all.shape[1])
eval_id_output = eval_id_all.reshape(eval_id_all.shape[0]*eval_id_all.shape[1])
print(val_id_output.shape, eval_id_output.shape)

(5320,) (5320,)


In [ ]:
val_dic = {'id':val_id_output}
output_cols = ['id']
for i in range(28):
  val_dic['F'+str(i+1)] = y_val_output[:,i]
  output_cols.append("F"+str(i+1))

val_df = pd.DataFrame(val_dic, columns=output_cols)
val_df

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOUSEHOLD_1_001_CA_1_validation,0.313865,0.253041,0.287952,0.288897,0.321564,0.452229,0.400139,0.300732,0.272612,0.307432,0.306493,0.341752,0.492320,0.455486,0.335184,0.276816,0.301210,0.294435,0.332897,0.487841,0.459854,0.325020,0.261636,0.290950,0.291902,0.331177,0.489789,0.469966
1,HOUSEHOLD_1_001_CA_2_validation,0.190069,0.171273,0.180496,0.185426,0.184442,0.187585,0.196693,0.178744,0.170099,0.178619,0.184730,0.183668,0.206053,0.206644,0.181645,0.167734,0.176151,0.182720,0.180882,0.198922,0.203453,0.177602,0.165898,0.175125,0.181215,0.179275,0.196023,0.205218
2,HOUSEHOLD_1_001_CA_3_validation,0.684742,0.597750,0.626201,0.638002,0.699721,0.863365,0.810473,0.677783,0.635813,0.666276,0.672443,0.729431,0.908779,0.864598,0.717917,0.643852,0.656922,0.650106,0.714189,0.900381,0.868617,0.705508,0.621086,0.639123,0.645664,0.709964,0.903163,0.884608
3,HOUSEHOLD_1_001_CA_4_validation,0.187764,0.168450,0.179319,0.184937,0.184838,0.203561,0.199234,0.177063,0.169002,0.180538,0.185816,0.185618,0.229761,0.214578,0.181658,0.167206,0.177835,0.183028,0.182724,0.223531,0.211431,0.178702,0.164526,0.175624,0.181373,0.180983,0.222396,0.216038
4,HOUSEHOLD_1_001_TX_1_validation,0.154877,0.146925,0.152373,0.160563,0.155674,0.143992,0.158258,0.152334,0.146752,0.152364,0.159593,0.155290,0.144233,0.159621,0.153448,0.144518,0.151388,0.157339,0.153485,0.142883,0.159120,0.151345,0.144435,0.149872,0.156121,0.151908,0.142927,0.160967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5315,HOUSEHOLD_1_541_TX_2_validation,2.306564,1.959483,2.060940,2.136796,2.305659,2.800772,2.716015,2.270417,2.011988,2.083515,2.147897,2.353212,2.856070,2.741107,2.364835,1.955447,2.114297,2.117593,2.326324,2.814456,2.780484,2.304946,1.956305,2.064241,2.062316,2.262535,2.771905,2.761107
5316,HOUSEHOLD_1_541_TX_3_validation,1.327712,1.042239,1.110516,1.169199,1.350146,1.873343,1.807311,1.323586,1.093713,1.137622,1.197920,1.438804,2.069441,1.880932,1.386239,1.068781,1.161231,1.208691,1.433751,2.004256,1.897213,1.354511,1.036909,1.112255,1.121722,1.319901,1.881329,1.828560
5317,HOUSEHOLD_1_541_WI_1_validation,1.201638,0.876181,0.950543,0.998101,1.211435,1.844696,1.690687,1.232792,0.945267,0.964891,1.053023,1.300906,1.956668,1.867662,1.299110,0.913217,0.989278,1.075044,1.252890,2.023439,1.980338,1.222900,0.887146,0.951103,0.997601,1.205365,1.912397,1.773924
5318,HOUSEHOLD_1_541_WI_2_validation,1.760200,1.364930,1.473108,1.493047,1.769464,2.496618,2.299774,1.782080,1.435144,1.473939,1.550473,1.850346,2.587358,2.475615,1.846708,1.394462,1.499611,1.549328,1.793446,2.662892,2.563837,1.756895,1.371342,1.455950,1.475284,1.755907,2.539293,2.367064


In [ ]:
eval_dic = {'id':eval_id_output}

for i in range(28):
  eval_dic['F'+str(i+1)] = y_eval_output[:,i]

eval_df = pd.DataFrame(eval_dic, columns=output_cols)
eval_df

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOUSEHOLD_1_001_CA_1_evaluation,0.317439,0.254271,0.287951,0.290634,0.327072,0.488271,0.472720,0.329366,0.273115,0.307694,0.317943,0.354641,0.508301,0.477995,0.335184,0.248347,0.281402,0.301938,0.332897,0.494716,0.468045,0.325020,0.249295,0.290950,0.297013,0.331177,0.489789,0.486182
1,HOUSEHOLD_1_001_CA_2_evaluation,0.180397,0.167681,0.176737,0.182826,0.181309,0.208698,0.210852,0.183047,0.168653,0.177168,0.186179,0.185798,0.214052,0.208479,0.181645,0.162247,0.171980,0.185594,0.180882,0.204081,0.205035,0.177602,0.162699,0.175125,0.183271,0.179275,0.196023,0.209253
2,HOUSEHOLD_1_001_CA_3_evaluation,0.693742,0.603916,0.628673,0.640254,0.703700,0.902115,0.888965,0.711797,0.637754,0.667404,0.686007,0.740699,0.925227,0.888744,0.717917,0.606898,0.633046,0.658169,0.714189,0.908839,0.879684,0.705508,0.605335,0.639123,0.651415,0.709964,0.903163,0.899816
3,HOUSEHOLD_1_001_CA_4_evaluation,0.179265,0.165445,0.176817,0.182672,0.182486,0.231534,0.222958,0.182382,0.167762,0.179199,0.186993,0.187282,0.242305,0.227527,0.181658,0.161000,0.171882,0.185370,0.182724,0.230202,0.216521,0.178702,0.161031,0.175624,0.183064,0.180983,0.222396,0.229137
4,HOUSEHOLD_1_001_TX_1_evaluation,0.152093,0.145719,0.150520,0.157598,0.153521,0.145523,0.161536,0.153830,0.145881,0.151363,0.161153,0.157838,0.145263,0.160461,0.153448,0.144164,0.148793,0.160995,0.153485,0.144154,0.160417,0.151345,0.144336,0.149872,0.159653,0.151908,0.142927,0.163363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5315,HOUSEHOLD_1_541_TX_2_evaluation,2.306127,2.010990,2.067611,2.102951,2.308381,2.843592,2.784601,2.328184,2.016210,2.087184,2.024443,2.247478,2.871002,2.771780,2.364835,1.864599,2.062927,2.051168,2.326324,2.802501,2.774621,2.304946,1.918648,2.064241,2.051461,2.262535,2.771905,2.649847
5316,HOUSEHOLD_1_541_TX_3_evaluation,1.317572,1.062788,1.104921,1.139820,1.344519,1.943928,1.884638,1.356010,1.091679,1.135291,1.113713,1.346848,2.057796,1.880910,1.386239,1.021558,1.141836,1.172126,1.433751,1.971275,1.873756,1.354511,1.037664,1.112255,1.118752,1.319901,1.881329,1.682813
5317,HOUSEHOLD_1_541_WI_1_evaluation,1.206170,0.904320,0.940739,1.006488,1.236820,1.905599,1.766414,1.260904,0.950282,0.968580,0.951968,1.205395,1.939606,1.866407,1.299110,0.863928,0.966232,1.028806,1.252890,1.982988,1.950344,1.222900,0.886483,0.951103,0.984873,1.205365,1.912397,1.602298
5318,HOUSEHOLD_1_541_WI_2_evaluation,1.763436,1.406890,1.448561,1.501265,1.809448,2.540742,2.378388,1.826803,1.440306,1.478015,1.419835,1.724055,2.572042,2.480974,1.846708,1.311003,1.461385,1.485564,1.793446,2.620037,2.532838,1.756895,1.353571,1.455950,1.456247,1.755907,2.539293,2.186005


The val_df and eval_df above are in the requested format for Kaggle competition predictions. In the end, we will merge these dataframes into a final consolidated file.

In [ ]:
val_df.to_csv("HOUSEHOLD_dept_1_validation.csv", index =False)
eval_df.to_csv("HOUSEHOLD_dept_1_evaluation.csv", index =False)
files.download("HOUSEHOLD_dept_1_validation.csv")
files.download("HOUSEHOLD_dept_1_evaluation.csv")